# Basic Models 

In [165]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import re
import sklearn
import string
import numpy as np 
import pandas as pd 
import seaborn as sn  
import matplotlib
import matplotlib.pyplot as plt 
from collections import Counter
import operator


## Import Data

In [88]:
df_impressions = pd.read_csv('impressions-train.csv')
df_ratings = pd.read_csv('ratings-final.csv')
df_test = pd.read_csv('test.csv')

In [18]:
df_test.head()
df_test.shape

,reviewerid,movie-code
0,45,57
1,235,185
2,507,106
3,170,36
4,469,115


(10000, 2)

In [37]:
df_impressions.head()
df_impressions.shape
len(Counter(df_impressions.reviewerid))
movies = Counter(df_impressions['movie-code'])
df_impressions.rating.mean()

df_ratings.head()
df_ratings.shape
len(Counter(df_ratings.reviewerid))
len(Counter(df_ratings['movie-code']))
df_ratings.rating.mean()
##ratings average > impressions average

,reviewerid,movie-code,rating
0,116,150,0
1,328,56,2
2,470,149,1
3,566,51,0
4,181,181,0


(27165, 3)

573

0.6265783176882017

,reviewerid,movie-code,rating
0,112,30,2
1,563,197,1
2,207,164,2
3,374,191,2
4,258,34,2


(44893, 3)

573

200

1.5642527788296616

In [52]:
#needed to create DF
movies = Counter(df_impressions['movie-code'])
avgs = df_impressions.groupby(['movie-code']).mean().to_dict()

In [ ]:
df_weighted_avg = pd.DataFrame().from_dict(movies, orient='index').reset_index()
df_weighted_avg = df_weighted_avg.rename(columns = {'index': 'movie_id', 0: 'imp_vote_count'})
df_weighted_avg['imp_vote_avg'] = df_weighted_avg['movie_id'].map(avgs['rating'])
#df_weighted_avg['movies'] = movies 
#df_weighted_avg['imp_vote_count'] = df_weighted_avg['movie'].map(z1)

In [70]:
review_id = Counter(df_impressions['reviewerid'])
review_r = dict(Counter(df_ratings['reviewerid']))

In [71]:
df_master = pd.DataFrame().from_dict(review_id, orient = 'index').reset_index()
df_master = df_master.rename(columns = {'index': 'review_id', 0: 'impression_count'})
df_master['ratings_count'] = df_master['review_id'].map(review_r)

In [93]:
Counter(df_impressions.duplicated())
Counter(df_ratings.duplicated())
df_impressions = df_impressions.drop_duplicates()
df_ratings = df_ratings.drop_duplicates()
Counter(df_impressions.duplicated())
Counter(df_ratings.duplicated())
df_merged = df_impressions.append(df_ratings)

Counter({False: 27154})

Counter({False: 44839})

Counter({False: 27154})

Counter({False: 44839})

In [106]:
Counter(df_merged.duplicated())
df_merged = df_merged.drop_duplicates()
df_merged.columns.values

Counter({False: 71990})

array(['reviewerid', 'movie-code', 'rating'], dtype=object)

In [155]:
t = df_merged.groupby('reviewerid').mean().to_dict()
s = df_merged.groupby('reviewerid').std().to_dict()
mins = df_merged.groupby('reviewerid').min().to_dict()
maxs = df_merged.groupby('reviewerid').max().to_dict()

In [156]:
df_merged['avgs'] = df_merged['reviewerid'].map(t['rating'])
df_merged['std'] = df_merged['reviewerid'].map(s['rating'])
df_merged['min'] = df_merged['reviewerid'].map(mins['rating'])
df_merged['max'] = df_merged['reviewerid'].map(maxs['rating'])

In [158]:
df_merged.tail()

,reviewerid,movie-code,rating,avgs,std,min,max
44888,35,16,2,1.340909,0.808632,0,2
44889,152,102,2,1.165468,0.839229,0,2
44890,496,30,2,1.487603,0.696607,0,2
44891,321,10,1,1.333333,0.735700,0,2
44892,487,156,2,1.460227,0.820180,0,2


In [129]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.model_selection import cross_val_score
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [159]:
X = df_merged[['reviewerid', 'movie-code', 'avgs', 'std', 'min', 'max']] # Features
y = df_merged.rating # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 70% training and 30% test
X_train.shape
X_test.shape


(57592, 6)

(14398, 6)

### Decision Tree

In [160]:
clf = DecisionTreeClassifier()
print(cross_val_score(clf, X, y, cv=5))



[0.42940482 0.45818864 0.47089874 0.46784276 0.44030006]


In [161]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

y_pred_train = clf.predict(X_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [162]:
print("Accuracy Train:",metrics.accuracy_score(y_train, y_pred_train))

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

Accuracy Train: 0.9996874565911932
Accuracy: 0.4679122100291707


array([[1499, 1100, 1180],
       [1033, 1378, 1586],
       [1193, 1569, 3860]])

#### Random Forest 

In [163]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred = rf.predict(X_test)
# Train the model on training data
print("Accuracy Train:",metrics.accuracy_score(y_train, y_pred_train))

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
metrics.confusion_matrix(y_test, y_pred)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

Accuracy Train: 0.9996874565911932
Accuracy: 0.437838588692874


array([[1277, 1058, 1444],
       [1012, 1274, 1711],
       [1334, 1535, 3753]])

In [66]:
df_weighted_avg

,movie_id,imp_vote_count,imp_vote_avg
0,1,223,1.152466
1,2,109,0.311927
2,3,186,0.736559
3,4,231,0.653680
4,5,191,0.931937
...,...,...,...
195,196,118,1.084746
196,197,151,0.655629
197,198,60,0.583333
198,199,144,1.180556


,movie_id,imp_vote_count
0,1,223
1,2,109
2,3,186
3,4,231
4,5,191
...,...,...
195,196,118
196,197,151
197,198,60
198,199,144


In [ ]:
V = movies_clean['vote_count']
R = movies_clean['vote_average']
C = movies_clean['vote_average'].mean()
m = movies_clean['vote_count'].quantile(0.70)

In [24]:
df_weighted_avg.shape

(200, 1)